<a href="https://colab.research.google.com/github/SufiyanShamsi/LangChain/blob/main/HuggingFaceHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* Using the HuggingFace API to import models   
* Creating an AutoSQL chain/"agent"



In [3]:
!pip install langchain langchain-community sqlalchemy python-dotenv
!pip install -qU langchain-huggingface huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [4]:
import os
from google.colab import userdata, files
from langchain_community.utilities import SQLDatabase
from langchain_community.llms import HuggingFaceHub
from langchain.chains import create_sql_query_chain #to create the AutoSQL chain
from sqlalchemy import create_engine
import sqlite3
from dotenv import load_dotenv

In [5]:
hf_token = userdata.get('hf_api_token')

In [6]:
os.environ["hf_api_token"] = userdata.get('hf_api_token')

In [7]:
DB_FILE = "chinook.db"

In [18]:
repo_id="codellama/CodeLlama-7b-Instruct-hf"

In [19]:
huggingface_llm = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={"temperature": 0.1, "max_new_tokens": 250}, # Flan-T5 can be sensitive to max_length
    huggingfacehub_api_token=hf_token,
)

In [20]:
engine = create_engine(f"sqlite:///{DB_FILE}")
chinbook_db = SQLDatabase(engine=engine)

In [21]:
from langchain_community.agent_toolkits import create_sql_agent
autosql_agent = create_sql_agent(
    llm=huggingface_llm,
    db=chinbook_db,
    agent_type="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

In [22]:
question1 = "List the names of the tables in the database."
print(f"\nQuestion: {question1}")
response1 = autosql_agent.invoke({"input": question1})
print("\nAnswer:")
print(response1["output"])


Question: List the names of the tables in the database.


> Entering new SQL Agent Executor chain...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: (Request ID: Root=1-680f86a1-3ac01aa11b20931e20b481f2;e0907a04-cd43-4422-b683-675920abab89)

403 Forbidden: None.
Cannot access content at: https://router.huggingface.co/hf-inference/models/codellama/CodeLlama-7b-Instruct-hf.
Make sure your token has the correct permissions.
The model codellama/CodeLlama-7b-Instruct-hf is too large to be loaded automatically (13GB > 10GB).